In [ ]:
# This notebook is used for creating synthetic users for training and testing the Q-learning agent 

In [1]:
!pip install sdv

In [2]:
import pandas as pd # for user survey data 
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer
import pickle 

C:\Users\hanna\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning:

Blowfish has been deprecated



In [3]:
# Load the user survey data 
user_df = pd.read_csv("Cluster1_Survey_Data.csv")
user_df.head()

,User,Weight-based,Calisthenics,Low-impact,Yoga,Pilates,Barre,Mobility,Flexibility,Core,...,Track Running,HIIT,Circuit,CrossFit,Cycling,Skipping,Swimming,Elliptical,Rowing,Stair Climb
0,1,0.9,0.7,0.6,0.4,0.2,0.2,0.4,0.2,0.3,...,0.7,0.8,0.6,0.8,0.7,0.7,0.6,0.9,0.9,0.8
1,2,0.9,0.9,0.7,0.1,0.2,0.4,0.3,0.4,0.6,...,0.6,0.7,0.8,0.7,0.6,0.7,0.4,0.8,0.9,0.9
2,3,0.9,0.6,0.6,0.1,0.3,0.3,0.2,0.2,0.3,...,0.6,0.6,0.6,0.6,0.6,0.5,0.5,0.8,0.8,0.9
3,4,0.9,0.7,0.6,0.1,0.2,0.4,0.2,0.3,0.3,...,0.6,0.9,0.7,0.7,0.7,0.7,0.5,0.5,0.4,0.6
4,5,0.9,0.7,0.7,0.4,0.5,0.5,0.3,0.2,0.5,...,0.6,0.8,0.7,0.7,0.8,0.6,0.4,0.9,0.9,0.9


In [4]:
# Define metadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=user_df)

#Fit SDV model
synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(user_df)

# Generate synthetic user data
synthetic_user_df = synthesizer.sample(413) # 310 for training; 103 for testing 

# Display first 5 rows
print(synthetic_user_df.head())

# Convert user data to list of user dictionaries 
simulated_users = []

for _, row in synthetic_user_df.iterrows():
    response_probs = {col:row[col] for col in synthetic_user_df.columns if col != "user_id"}
    user = {"response_probs": response_probs}
    simulated_users.append(user)


C:\Users\hanna\anaconda3\Lib\site-packages\sdv\single_table\base.py:162: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

C:\Users\hanna\anaconda3\Lib\site-packages\sdv\single_table\base.py:128: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



   User  Weight-based  Calisthenics  Low-impact  Yoga  Pilates  Barre  \
0     5           0.9           0.7         0.7   0.4      0.5    0.5   
1     2           0.9           0.7         0.6   0.4      0.2    0.3   
2     3           0.9           0.7         0.6   0.1      0.2    0.4   
3     2           0.9           0.8         0.6   0.1      0.2    0.3   
4     3           0.9           0.9         0.7   0.1      0.2    0.4   

   Mobility  Flexibility  Core  ...  Track Running  HIIT  Circuit  CrossFit  \
0       0.3          0.2   0.3  ...            0.7   0.7      0.7       0.7   
1       0.4          0.2   0.3  ...            0.7   0.9      0.7       0.7   
2       0.3          0.2   0.3  ...            0.7   0.8      0.7       0.7   
3       0.4          0.2   0.3  ...            0.7   0.8      0.8       0.7   
4       0.3          0.3   0.3  ...            0.7   0.7      0.8       0.7   

   Cycling  Skipping  Swimming  Elliptical   Rowing   Stair Climb  
0      0.8       0

In [5]:
# Save the simulated user data to a pickle file 
with open("Simulated_Users_Cluster_1.pickle", "wb") as f:
    pickle.dump(simulated_users, f)

In [6]:
# Load the saved pickle file
with open("Simulated_Users_Cluster_1.pickle", "rb") as f:
    loaded_users = pickle.load(f)

# Convert to DataFrame and save as CSV
import pandas as pd
df = pd.DataFrame([user["response_probs"] for user in loaded_users])
df.to_csv("Simulated_Users_Cluster_1_Expanded.csv", index=False)